In [10]:
import nbimporter
import LIA_Tutorial_Utils
import io
import sys
import math
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # graphs/plotting
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt
from sklearn.preprocessing import LabelEncoder
from scikitplot.metrics import plot_roc, plot_confusion_matrix
from sklearn.metrics import classification_report, balanced_accuracy_score
from sklearn.isotonic import IsotonicRegression
from sklearn.metrics import log_loss
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.13.0


In [28]:
def load_data():
    
    x_train = pd.read_csv('Data_sets\X_train_preprocessed.csv')
    x_valid = pd.read_csv('Data_sets\X_valid_preprocessed.csv')
    x_test = pd.read_csv('Data_sets\X_test_preprocessed.csv')
    y_train = pd.read_csv('Data_sets\Y_train.csv')
    y_valid = pd.read_csv('Data_sets\Y_valid.csv')
    y_test = pd.read_csv('Data_sets\Y_test.csv')
    x_train = x_train.set_index('Id')
    x_valid = x_valid.set_index('Id')
    x_test = x_test.set_index('Id')
    y_train = y_train.set_index('Id')
    y_valid = y_valid.set_index('Id')
    y_test = y_test.set_index('Id')
    x_full=pd.concat([x_train, x_valid, x_test])
    y_full=pd.concat([y_train, y_valid, y_test])
    x_full=x_full.sort_index()
    y_full=y_full.sort_index()
    # Convert from pandas to numpy
    x_train=x_train.to_numpy().astype('float32')
    x_valid=x_valid.to_numpy().astype('float32')
    x_test=x_test.to_numpy().astype('float32')

    
    return x_full, y_full, x_train, x_valid, x_test, y_train, y_valid, y_test

In [29]:
def evaluate_first_tuner(search_space_summary):    
    expected_search_space = """
Search space summary
Default search space size: 4
hidden_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 5, 'step': 1, 'sampling': 'linear'}
activations (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'sigmoid', 'tanh'], 'ordered': False}
learning_rate (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'linear'}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 15, 'max_value': 120, 'step': 5, 'sampling': 'linear'}    
    """
    
    # Compare the generated search space summary with the expected search space
    if search_space_summary.strip() == expected_search_space.strip():
        print_value = search_space_summary + "\nSearch space matches expected."
        print("\033[92m"+print_value)
    else:
        print_value= search_space_summary + "\n\nSearch space does not match expected. Please review."
        print("\033[91m"+print_value)
    

In [30]:
def evaluate_final_tuner_1(search_space_summary):    
    expected_search_space = """
Search space summary
Default search space size: 2
learning_rate (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}
batch_size (Choice)
{'default': 16, 'conditions': [], 'values': [16, 32, 64, 128, 256, 512], 'ordered': True}
"""
    
    # Compare the generated search space summary with the expected search space
    if search_space_summary.strip() == expected_search_space.strip():
        print_value = search_space_summary + "\nSearch space matches expected."
        print("\033[92m"+print_value)

    else:
        print_value= search_space_summary + "\n\nSearch space does not match expected. Please review.\n"
        print("\033[91m"+print_value)
    

In [24]:
def evaluate_final_tuner_2(search_space_summary, best_lr, best_bs):   
    expected_search_space = """
Search space summary
Default search space size: 4
learning_rate (Fixed)
{'conditions': [], 'value': """+ str(best_lr)+"""}
batch_size (Fixed)
{'conditions': [], 'value': """+ str(best_bs)+"""}
hidden_layers (Int)
{'default': 1, 'conditions': [], 'min_value': 1, 'max_value': 5, 'step': 1, 'sampling': 'linear'}
units_1 (Int)
{'default': 21, 'conditions': [], 'min_value': 15, 'max_value': 120, 'step': 5, 'sampling': 'linear'}
"""
    
    # Compare the generated search space summary with the expected search space
    if search_space_summary.strip() == expected_search_space.strip():
        print_value = search_space_summary + "\nSearch space matches expected."
        print("\033[92m"+print_value)

    else:
        print_value= search_space_summary + "\n\nSearch space does not match expected. Please review.\n"+ expected_search_space
        print("\033[91m"+print_value)
    